In [32]:
"""Implementation based on the template of ALIGNN."""

import random
from pathlib import Path
from typing import Optional

# from typing import Dict, List, Optional, Set, Tuple

from mp_api.client import MPRester
import torch

import os
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd
from jarvis.core.atoms import Atoms
import graph
#from ComFormerFork.graph import PygGraph, PygStructureDataset
from jarvis.db.figshare import data as jdata
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
from jarvis.db.jsonutils import dumpjson
# from sklearn.pipeline import Pipeline
import pickle as pk
from sklearn.preprocessing import StandardScaler
# use pandas progress_apply
tqdm.pandas()

load_dotenv()
load_dotenv(Path("/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/matprojectapi.env"))
API_KEY = os.getenv("MP_API_KEY")

dft3d = jdata("dft_3d")               
df = pd.DataFrame(dft3d)     

# print(df.columns)
# print(type(dft3d[0]))
# try:
#     print(dft3d[0].keys())          # if it’s a dict-like
# except:
#     print(vars(dft3d[0]))           # if it’s an object with attrs

# from jarvis.db.restapi import Api

# mp_map = []
# for jid in df["jid"]:
#     entry = Api.get_data_by_template_id("dft_3d", jid)
#     # entry.db_ids is guaranteed to be a dict if it exists
#     mp_map.append({
#         "jid": jid,
#         "mpid": entry.db_ids.get("materialsproject")  # maybe None
#     })
# mp_df = pd.DataFrame(mp_map)

# df = df.merge(mp_df, on="jid", how="left")

# 3) (Optional) drop any rows where mpid is missing
#df = df.dropna(subset=["mpid"]).reset_index(drop=True)


# df["mpid"] = df["db_ids"].apply(lambda x: x.get("materialsproject") if isinstance(x, dict) else None)
# df = df.dropna(subset=["mpid"])        # keep only entries with an MP match


# with MPRester(api_key = API_KEY) as m:
#     docs = m.materials.summary.search(
#         jid__in=df["jid"].tolist(),
#         fields=["jid","topological_type"]
#     )

#optimade
import pandas as pd
import requests
import time

# --- Start: MODIFIED Definition of jarvisdft_optimade ---
def jarvisdft_optimade(
    prefix="https://jarvis.nist.gov/optimade/jarvisdft/v1/structures/?filter=",
    query="elements HAS ALL C,Si",
):
    url = prefix + query
    vals = []
    print(f"Querying URL: {url}")
    current_url_for_error = url # Store initial URL for error reporting

    while True:
        if url is not None:
            current_url_for_error = url # Update for current page
            try:
                response = requests.get(url, timeout=30)
                response.raise_for_status() # Check for HTTP errors (4xx or 5xx)

                json_response = None # Initialize
                if not response.content: # Check if response body is empty
                    print(f"  Warning: Empty response from {url}")
                else:
                    try:
                        json_response = response.json()
                    except requests.exceptions.JSONDecodeError as je:
                        print(f"  Error decoding JSON from {url}: {je}")
                        print(f"  Response text: {response.text[:500]}...")
                
                if json_response is None:
                    print(f"  Could not parse JSON or response was empty/problematic for {url}. Stopping pagination for this query.")
                    break # Break from the while True loop

                data = json_response.get("data", []) # Safely get data, defaults to []
                for i in data:
                    vals.append(i)
                
                # Safely get the 'next' link for pagination
                links_object = json_response.get("links", {})
                if isinstance(links_object, dict):
                    url = links_object.get("next")
                else: # links object is not a dictionary or is missing
                    url = None 
                
                if url:
                    print(f"  Fetching next page: {url}")

            except requests.exceptions.RequestException as e:
                print(f"  Request error for {current_url_for_error}: {e}")
                break # Stop if there's a request error
            except Exception as e_inner: # Catch any other unexpected error within the try
                print(f"  Unexpected error processing {current_url_for_error}: {e_inner}")
                break
        else: # url is None, so no more pages
            break
        
        # Be polite to the API, even if a page was problematic but we continue
        time.sleep(0.2) 
    return vals
# --- End: Definition of jarvisdft_optimade ---

# Simulate your initial DataFrame with 'jid' column
data = {'jid': ['JVASP-1', 'JVASP-2', 'JVASP-1002', 'JVASP-99999']} # Example JIDs
# For testing with a JID that might exist and have an MPID cross-reference:
# data = {'jid': ['JVASP-1063']} # Example: JVASP-1063 (Si) is mp-149
df = pd.DataFrame(data)

# --- batch all JIDs into one OPTIMADE filter, request up to 100 results at once ---
jids = df["jid"].tolist()
or_filters = " OR ".join(f'(immutable_id="{jid}")' for jid in jids)
base_url = "https://jarvis.nist.gov/optimade/jarvisdft/v1/structures/"
url = f"{base_url}?page_limit=100&filter={or_filters}"

resp = requests.get(url, timeout=30)
resp.raise_for_status()
entries = resp.json().get("data", [])

# build a dict: JARVIS ID → MPID
mp_map = {}
for e in entries:
    attrs = e.get("attributes", {})
    for db_ref in attrs.get("external_databases", []):
        if db_ref.get("name", "").lower() == "materials project":
            mp_map[e.get("id")] = db_ref.get("id")
            break

# turn it into a DataFrame and merge back into your df
optimade_mp_df = pd.DataFrame([
    {"jid": jid, "mpid_optimade": mp_map.get(jid)}
    for jid in jids
])
df_merged_optimade = df.merge(optimade_mp_df, on="jid", how="left")
#print(df_merged_optimade)

import pandas as pd
from mp_api.client import MPRester
import os
from dotenv import load_dotenv
from pathlib import Path

# --- (Assuming previous parts of your script are here, including API_KEY loading and df_merged_optimade creation) ---

# Load environment variables if not already done
# load_dotenv() # General .env
# load_dotenv(Path("/Users/abiralshakya/Documents/Research/Topological_Insulators_OnGithub/generative_nmti/Integrated_Magnetic_Topological/matprojectapi.env")) # Specific .env
# API_KEY = os.getenv("MP_API_KEY")

# df_merged_optimade should be a DataFrame with a column like 'mpid_optimade' containing Material IDs
# Example:
# data_example = {'jid': ['JVASP-1063'], 'mpid_optimade': ['mp-149']} # JVASP-1063 (Si) is mp-149
# df_merged_optimade = pd.DataFrame(data_example)

mp_ids_raw = df_merged_optimade["mpid_optimade"].dropna().tolist()
# Ensure all IDs are strings and filter out any potential non-string/None types that dropna might miss if column isn't strictly strings
mp_ids = [str(mid) for mid in mp_ids_raw if mid is not None and str(mid).startswith("mp-")]

label_df = pd.DataFrame(columns=["mpid", "topo_type", "is_topo"]) # Initialize an empty DataFrame

if not API_KEY:
    print("Error: MP_API_KEY is not set. Please check your .env configuration.")
elif not mp_ids:
    print("No valid Material IDs found to query for topological information.")
else:
    try:
        with MPRester(API_KEY) as m:
            # Use the TopologyRester to get topological information
            # Fields in TopologyDoc include 'type_of_topology_no_vdw' and 'type_of_topology_vdw'
            # Choose the one relevant to your study, or fetch both.
            # For this example, we'll use 'type_of_topology_no_vdw'.
            docs = m.topology.search(
                material_ids=mp_ids,
                fields=["material_id", "type_of_topology_no_vdw"] # Correct fields for topology
            )

        # Process the results
        if docs:
            processed_docs = []
            for d in docs:
                # Ensure the document (d) and its attributes exist before accessing
                mpid = getattr(d, 'material_id', None)
                topo_type_value = getattr(d, 'type_of_topology_no_vdw', None) # Or type_of_topology_vdw

                if mpid is not None: # Only proceed if material_id is present
                    processed_docs.append({
                        "mpid": str(mpid), # Ensure mpid is a string for consistent merging
                        "topo_type": topo_type_value
                    })
            
            if processed_docs:
                 label_df = pd.DataFrame(processed_docs)

                 # Define "good" topological types
                 good = {"Z2_INSULATOR", "CRYSTALLINE_INSULATOR"} # These values should match potential outputs of type_of_topology_no_vdw
                 label_df["is_topo"] = label_df["topo_type"].isin(good).astype(int)
            else:
                print("No topological data returned for the given material IDs.")

        else:
            print("No documents returned from topology search.")

    except Exception as e:
        print(f"An error occurred during MPRester query or processing: {e}")
        # label_df remains empty or as previously defined if error occurs

# --- Merging back with your main DataFrame ---
# Ensure the column names for merging are correct.
# df_merged_optimade has 'mpid_optimade'. label_df has 'mpid'.
if not label_df.empty:
    df_merged_optimade = df_merged_optimade.merge(
        label_df[["mpid", "is_topo", "topo_type"]], # Include "topo_type" if you need it later
        left_on="mpid_optimade",
        right_on="mpid",
        how="left"
    )
    # Fill NaN values for 'is_topo' that might result from materials not found or not having topology data
    df_merged_optimade["is_topo"] = df_merged_optimade["is_topo"].fillna(0).astype(int)
    if "topo_type_y" in df_merged_optimade.columns : # if topo_type already existed. _y is from merge
        df_merged_optimade.rename(columns={"topo_type_y": "topo_type"}, inplace=True)
    elif "topo_type_x" in df_merged_optimade.columns and "topo_type" in df_merged_optimade.columns: # if topo_type was original, and new is topo_type
        # decide which one to keep or rename appropriately
        pass
    elif "topo_type" not in df_merged_optimade.columns and "topo_type_y" not in df_merged_optimade.columns and "topo_type_x" not in df_merged_optimade.columns:
         # If 'topo_type' wasn't in the original df_merged_optimade and label_df was empty or didn't merge
         # we might need to add it if it's expected.
         if "mpid" in label_df.columns: # only if label_df had mpid to begin with
            df_merged_optimade["topo_type"] = df_merged_optimade["mpid"].map(label_df.set_index("mpid")["topo_type"])


else: # If label_df is empty (e.g., no mp_ids or API error)
    df_merged_optimade["is_topo"] = 0
    df_merged_optimade["topo_type"] = None # Or np.nan

# print("\n--- DataFrame after merging topological data ---")
# print(df_merged_optimade.head())

# The rest of your script (load_dataset, etc.) would follow

#optimade_mp_df = pd.DataFrame(mp_map_optimade)
# Merge with the original df. df_merged_optimade will have 'jid' and 'mpid_optimade'
#df_merged_optimade = df.merge(optimade_mp_df, on="jid", how="left")

print("\n--- Merged DataFrame (from OPTIMADE) ---")
print(df_merged_optimade)


# mp_ids = df_merged_optimade["mpid_optimade"].dropna().tolist()

# with MPRester(API_KEY) as m:
#     docs = m.materials.summary._search(
#         suburl="atlas/topology",
#         material_ids=",".join(mp_ids),
#         fields=["material_id", "topological_type"],
#     )
#     # docs = m.materials.summary.search(
#     #     material_ids= mp_ids, 
#     #     fields = ["material_id", "topological_type"]
#     # )
#     # docs = m.materials.summary.search(
#     #     material_ids=df["mpid"].tolist(),
#     #     fields=["material_id","topological_type"]
#     # )


# label_df = pd.DataFrame([{
#     "mpid": d.material_id,
#     "topo_type": d.topological_type
# } for d in docs])

# # map to binary
# good = {"Z2_INSULATOR","CRYSTALLINE_INSULATOR"}
# label_df["is_topo"] = label_df["topo_type"].isin(good).astype(int)

# df = df.merge(label_df[["mpid","is_topo"]], on="mpid", how="left")
# df["is_topo"] = df["is_topo"].fillna(0).astype(int)

def load_dataset(
    name: str = "dft_3d",
    target=None,
    limit: Optional[int] = None,
    classification_threshold: Optional[float] = None,
):
    """Load jarvis data."""
    d = jdata(name)
    data = []
    for i in d:
        if i[target] != "na" and not math.isnan(i[target]):
            if classification_threshold is not None:
                if i[target] <= classification_threshold:
                    i[target] = 0
                elif i[target] > classification_threshold:
                    i[target] = 1
                else:
                    raise ValueError(
                        "Check classification data type.",
                        i[target],
                        type(i[target]),
                    )
            data.append(i)
    d = data
    if limit is not None:
        d = d[:limit]
    d = pd.DataFrame(d)
    return d


def mean_absolute_deviation(data, axis=None):
    """Get Mean absolute deviation."""
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

def load_pyg_graphs(
    df: pd.DataFrame,
    name: str = "dft_3d",
    neighbor_strategy: str = "k-nearest",
    cutoff: float = 8,
    max_neighbors: int = 12,
    cachedir: Optional[Path] = None,
    use_canonize: bool = False,
    use_lattice: bool = False,
    use_angle: bool = False,
):
    """Construct crystal graphs.

    Load only atomic number node features
    and bond displacement vector edge features.

    Resulting graphs have scheme e.g.
    ```
    Graph(num_nodes=12, num_edges=156,
          ndata_schemes={'atom_features': Scheme(shape=(1,)}
          edata_schemes={'r': Scheme(shape=(3,)})
    ```
    """
    def atoms_to_graph(atoms):
        """Convert structure dict to DGLGraph."""
        structure = Atoms.from_dict(atoms)
        return PygGraph.atom_dgl_multigraph(
            structure,
            neighbor_strategy=neighbor_strategy,
            cutoff=cutoff,
            atom_features="atomic_number",
            max_neighbors=max_neighbors,
            compute_line_graph=False,
            use_canonize=use_canonize,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    
    graphs = df["atoms"].parallel_apply(atoms_to_graph).values
    # graphs = df["atoms"].apply(atoms_to_graph).values

    return graphs


def get_id_train_val_test(
    total_size=1000,
    split_seed=123,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    n_train=None,
    n_test=None,
    n_val=None,
    keep_data_order=False,
):
    """Get train, val, test IDs."""
    if (
        train_ratio is None
        and val_ratio is not None
        and test_ratio is not None
    ):
        if train_ratio is None:
            assert val_ratio + test_ratio < 1
            train_ratio = 1 - val_ratio - test_ratio
            print("Using rest of the dataset except the test and val sets.")
        else:
            assert train_ratio + val_ratio + test_ratio <= 1
    # indices = list(range(total_size))
    if n_train is None:
        n_train = int(train_ratio * total_size)
    if n_test is None:
        n_test = int(test_ratio * total_size)
    if n_val is None:
        n_val = int(val_ratio * total_size)
    ids = list(np.arange(total_size))
    if not keep_data_order:
        random.seed(split_seed)
        random.shuffle(ids)
    if n_train + n_val + n_test > total_size:
        raise ValueError(
            "Check total number of samples.",
            n_train + n_val + n_test,
            ">",
            total_size,
        )

    id_train = ids[:n_train]
    id_val = ids[-(n_val + n_test) : -n_test]
    id_test = ids[-n_test:]
    return id_train, id_val, id_test


def get_pyg_dataset(
    dataset=[],
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
    use_lattice=False,
    use_angle=False,
    data_from='Jarvis',
    use_save=False,
    mean_train=None,
    std_train=None,
    now=False, # for test
):
    """Get pyg Dataset."""
    df = pd.DataFrame(dataset)
    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    output_dir = "./saved_data/" + tmp_name + "test_graph_angle.pkl" # for fast test use
    print("data range", np.max(vals), np.min(vals))
    print(output_dir)
    print('graphs not saved')
    graphs = load_pyg_graphs(
        df,
        name=name,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        use_lattice=use_lattice,
        use_angle=use_angle,
    )
    if mean_train == None:
        mean_train = np.mean(vals)
        std_train = np.std(vals)
        data = graph.PygStructureDataset(
            df,
            graphs,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
        )
    else:
        data = graph.PygStructureDataset(
            df,
            graphs,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
        )
    return data, mean_train, std_train


def get_train_val_loaders(
    dataset: str = "dft_3d",
    dataset_array=[],
    target: str = "formation_energy_peratom",
    atom_features: str = "cgcnn",
    neighbor_strategy: str = "k-nearest",
    n_train=None,
    n_val=None,
    n_test=None,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    batch_size: int = 5,
    standardize: bool = False,
    line_graph: bool = True,
    split_seed: int = 123,
    workers: int = 0,
    pin_memory: bool = True,
    save_dataloader: bool = False,
    filename: str = "sample",
    id_tag: str = "jid",
    use_canonize: bool = False,
    cutoff: float = 8.0,
    max_neighbors: int = 12,
    classification_threshold: Optional[float] = None,
    target_multiplication_factor: Optional[float] = None,
    standard_scalar_and_pca=False,
    keep_data_order=False,
    output_features=1,
    output_dir=None,
    matrix_input=False,
    pyg_input=False,
    use_lattice=False,
    use_angle=False,
    use_save=True,
    mp_id_list=None,
):
    """Help function to set up JARVIS train and val dataloaders."""
    # data loading
    mean_train=None
    std_train=None
    assert (matrix_input and pyg_input) == False
    
    train_sample = filename + "_train.data"
    val_sample = filename + "_val.data"
    test_sample = filename + "_test.data"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if (
        os.path.exists(train_sample)
        and os.path.exists(val_sample)
        and os.path.exists(test_sample)
        and save_dataloader
    ):
        print("Loading from saved file...")
        print("Make sure all the DataLoader params are same.")
        print("This module is made for debugging only.")
        train_loader = torch.load(train_sample)
        val_loader = torch.load(val_sample)
        test_loader = torch.load(test_sample)
        if train_loader.pin_memory != pin_memory:
            train_loader.pin_memory = pin_memory
        if test_loader.pin_memory != pin_memory:
            test_loader.pin_memory = pin_memory
        if val_loader.pin_memory != pin_memory:
            val_loader.pin_memory = pin_memory
        if train_loader.num_workers != workers:
            train_loader.num_workers = workers
        if test_loader.num_workers != workers:
            test_loader.num_workers = workers
        if val_loader.num_workers != workers:
            val_loader.num_workers = workers
        print("train", len(train_loader.dataset))
        print("val", len(val_loader.dataset))
        print("test", len(test_loader.dataset))
        return (
            train_loader,
            val_loader,
            test_loader,
            train_loader.dataset.prepare_batch,
        )
    else:
        if not dataset_array:
            d = jdata(dataset)
        else:
            d = dataset_array
            # for ii, i in enumerate(pc_y):
            #    d[ii][target] = pc_y[ii].tolist()

        dat = []
        if classification_threshold is not None:
            print(
                "Using ",
                classification_threshold,
                " for classifying ",
                target,
                " data.",
            )
            print("Converting target data into 1 and 0.")
        all_targets = []

        # TODO:make an all key in qm9_dgl
        if dataset == "qm9_dgl" and target == "all":
            print("Making all qm9_dgl")
            tmp = []
            for ii in d:
                ii["all"] = [
                    ii["mu"],
                    ii["alpha"],
                    ii["homo"],
                    ii["lumo"],
                    ii["gap"],
                    ii["r2"],
                    ii["zpve"],
                    ii["U0"],
                    ii["U"],
                    ii["H"],
                    ii["G"],
                    ii["Cv"],
                ]
                tmp.append(ii)
            print("Made all qm9_dgl")
            d = tmp
        for i in d:
            if isinstance(i[target], list):  # multioutput target
                all_targets.append(torch.tensor(i[target]))
                dat.append(i)

            elif (
                i[target] is not None
                and i[target] != "na"
                and not math.isnan(i[target])
            ):
                if target_multiplication_factor is not None:
                    i[target] = i[target] * target_multiplication_factor
                if classification_threshold is not None:
                    if i[target] <= classification_threshold:
                        i[target] = 0
                    elif i[target] > classification_threshold:
                        i[target] = 1
                    else:
                        raise ValueError(
                            "Check classification data type.",
                            i[target],
                            type(i[target]),
                        )
                dat.append(i)
                all_targets.append(i[target])
    
    if mp_id_list is not None:
        if mp_id_list == 'bulk':
            print('using mp bulk dataset')
            with open('/data/keqiangyan/bulk_shear/bulk_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('/data/keqiangyan/bulk_shear/bulk_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('/data/keqiangyan/bulk_shear/bulk_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)
        
        if mp_id_list == 'shear':
            print('using mp shear dataset')
            with open('/data/keqiangyan/bulk_shear/shear_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('/data/keqiangyan/bulk_shear/shear_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('/data/keqiangyan/bulk_shear/shear_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

    else:
        id_train, id_val, id_test = get_id_train_val_test(
            total_size=len(dat),
            split_seed=split_seed,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            n_train=n_train,
            n_test=n_test,
            n_val=n_val,
            keep_data_order=keep_data_order,
        )
        ids_train_val_test = {}
        ids_train_val_test["id_train"] = [dat[i][id_tag] for i in id_train]
        ids_train_val_test["id_val"] = [dat[i][id_tag] for i in id_val]
        ids_train_val_test["id_test"] = [dat[i][id_tag] for i in id_test]
        dumpjson(
            data=ids_train_val_test,
            filename=os.path.join(output_dir, "ids_train_val_test.json"),
        )
        dataset_train = [dat[x] for x in id_train]
        dataset_val = [dat[x] for x in id_val]
        dataset_test = [dat[x] for x in id_test]
    
    train_data, mean_train, std_train = get_pyg_dataset(
        dataset=dataset_train,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="train_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
    )
    val_data,_,_ = get_pyg_dataset(
        dataset=dataset_val,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="val_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
    )
    test_data,_,_ = get_pyg_dataset(
        dataset=dataset_test,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="test_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
    )

    
    collate_fn = train_data.collate
    if line_graph:
        collate_fn = train_data.collate_line_graph

    # use a regular pytorch dataloader
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    test_loader = DataLoader(
        test_data,
        batch_size=1,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory,
    )
    if save_dataloader:
        torch.save(train_loader, train_sample)
        torch.save(val_loader, val_sample)
        torch.save(test_loader, test_sample)
    
    print("n_train:", len(train_loader.dataset))
    print("n_val:", len(val_loader.dataset))
    print("n_test:", len(test_loader.dataset))
    return (
        train_loader,
        val_loader,
        test_loader,
        train_loader.dataset.prepare_batch,
        mean_train,
        std_train,
    )
    

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
No valid Material IDs found to query for topological information.

--- Merged DataFrame (from OPTIMADE) ---
           jid mpid_optimade  is_topo topo_type
0      JVASP-1          None        0      None
1      JVASP-2          None        0      None
2   JVASP-1002          None        0      None
3  JVASP-99999          None        0      None
